In [6]:
def strcounter(s):
    slozhnost = 0
    for symbol in set(s):
        count = 0
        for sub_symbol in s:
            if symbol == sub_symbol:
                count += 1
            slozhnost += 1
        print(symbol, ":", count)
    return slozhnost
s = "aaabbbccddddddeeeeeee"
sl = strcounter(s)
n = len(s)
print(n, sl)

c : 2
e : 7
b : 3
d : 6
a : 3
21 105


In [20]:
A = [random.random() for i in range(100000)]

In [16]:
import time
def BubbleSort(A):
    for j in range(len(A) - 1):
        for i in range(len(A) - 1):
            if A[i] > A[i + 1]:
                A[i], A[i + 1] = A[i + 1], A[i]
    return A
t= time.time()
print(BubbleSort(A))
print(time.time()-t)

[9.505102340723326e-05, 0.00016578736130712368, 0.0003836886478074675, 0.00040467640506336444, 0.0004157509275954929, 0.0004780508411305062, 0.0004856240648283272, 0.0005324021559500736, 0.000626016462543566, 0.0007647972529576386, 0.000783470692024002, 0.0008439528554914366, 0.0009901725459404709, 0.0010982482301371554, 0.0011476054058073837, 0.001437429917161448, 0.0017763493684510578, 0.001833926649031814, 0.0019638761972106122, 0.0020758726867307065, 0.0021108448359481446, 0.002159756535829005, 0.0021672248666607663, 0.0022369298068358745, 0.0025907703616083744, 0.0025944817721141877, 0.002777613870722062, 0.0028151128836476635, 0.0028384989303478214, 0.0028507024786328383, 0.0029201182371998424, 0.0030165538241366807, 0.0033532759334793383, 0.003606804521956053, 0.003681351115104592, 0.0038132015486462834, 0.00386853289201905, 0.004113168406600476, 0.004148965518019687, 0.00441321711552245, 0.004451735674376844, 0.004687909584542904, 0.0047553967216654724, 0.005053265358041181, 0.

In [25]:
import random
import time
def qsort(A):
    if len(A)<=1:
        return A
    else:
        q = random.choice(A)
        s_nums = []
        m_nums = []
        e_nums = []
        for i in A:
            if i < q:
                s_nums.append(i)
            elif i > q:
                m_nums.append(i)
            else:
                e_nums.append(i)
        return qsort(s_nums) + e_nums + qsort(m_nums)
t= time.time()
print(qsort(A))
print(time.time()-t)

[4.496419030219911e-06, 9.9772647238483e-06, 1.5623531576580696e-05, 2.2002658073505543e-05, 3.6713870157578654e-05, 5.504078201745699e-05, 5.6306303257702695e-05, 6.513932294993641e-05, 7.369420033298546e-05, 9.04148637710378e-05, 9.285212111775909e-05, 0.00010225727739876511, 0.00011190718511910891, 0.00011846503638945904, 0.0001275141713514838, 0.00013190394528039828, 0.00013899685051388389, 0.00014138448171641116, 0.00016144088896241637, 0.00016953874370573274, 0.0001739509940108519, 0.00020677522870127252, 0.0002213559165659129, 0.00022612502624086428, 0.0002323473877694493, 0.00023934204185760422, 0.0002439529066731172, 0.00025521374469061886, 0.00028145758689157585, 0.0003209328375898268, 0.00032267279326902987, 0.00034844741940631696, 0.00036987206782435056, 0.0003880097094141055, 0.00039561028592882685, 0.0003989585039476662, 0.0004036364077935062, 0.00040981205087486927, 0.0004116231652541158, 0.0004141009556245523, 0.00041457451944770796, 0.0004217852907146824, 0.00042590843